# Color cycle analysis

The training process takes several hours.

In [1]:
import csv
import gzip
import itertools
import logging
import os
import random
import shutil
import sqlite3
import sys
import time
import numpy as np
import colorspacious
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, SeparableConv1D, Activation
import tensorflow as tf
import sklearn.utils

## Ensuring reproducibility

For reproducibility, a fully-deterministic training procedure is desired. Seeding the PRNG is not enough. As efforts to ensure reproducibility with GPU-based training (`TF_DETERMINISTIC_OPS=1`) were not successful, CPU-based training was used. Although the number of CPU threads did not seem to affect the results, a single thread was still used, since doing so is [recommended to ensure reproducibility](https://github.com/NVIDIA/framework-determinism/blob/28091e4fb1483685fc78b7ab844a5ae6ddf55a14/README.md#cpu).

In [2]:
# Force CPU operation for deterministic and reproducible results
# Disable parallelism per https://github.com/NVIDIA/framework-determinism/blob/28091e4fb1483685fc78b7ab844a5ae6ddf55a14/README.md#cpu
tf.config.set_visible_devices([], "GPU")
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

In [3]:
# Needed for deterministic results
SEED = 567687
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
print("Python", sys.version)
print("NumPy", np.__version__)
print("TensorFlow", tf.__version__)
print("Scikit-learn", sklearn.__version__)
print("Colorspacious", colorspacious.__version__)

Python 3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]
NumPy 1.19.5
TensorFlow 2.4.1
Scikit-learn 0.24.2
Colorspacious 1.1.2


## Configuration

In [5]:
DB_FILE = "../survey-results/results.db"

In [6]:
ALL_NUM_COLORS = [6, 8]
DATA_SPLIT_FRAC = 0.8
ENSEMBLE_COUNT = 100
NUM_EPOCHS = 120
BATCH_SIZE = 1024

## Color conversion and sorting functions

Data are all stored as 8-bit RGB values and need to be converted to CAM02-UCS.

We also need to be able to sort along the three CAM02-UCS axes.

In [7]:
def to_jab(color):
    """
    Convert hex color code (without `#`) to CAM02-UCS.
    """
    rgb = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color]
    jab = [colorspacious.cspace_convert(i, "sRGB255", "CAM02-UCS") for i in rgb]
    return np.array(jab)


def sort_colors_by_j(colors):
    """
    Sorts colors by CAM02-UCS J' axis.
    """
    return colors[np.lexsort(colors[:, ::-1].T, 0)]


def sort_colors_by_a(colors):
    """
    Sorts colors by CAM02-UCS a' axis.
    """
    return colors[np.argsort(colors[:, ::-1].T[1])]


def sort_colors_by_b(colors):
    """
    Sorts colors by CAM02-UCS b' axis.
    """
    return colors[np.argsort(colors[:, ::-1].T[2])]

## Data loading

Survey data are loaded from a SQLite database. The 8-bit RGB values are converted to CAM02-UCS and sorted along the three CAM02-UCS axes.

In [8]:
# Load survey data
cycle_data = {}
cycle_targets = {}
min_count = 1e10

conn = sqlite3.connect(DB_FILE)
c = conn.cursor()

for num_colors in ALL_NUM_COLORS:
    count = 0
    cycle_data[num_colors] = []
    cycle_targets[num_colors] = []
    for row in c.execute(
        f"SELECT c1, c2, o, cp, sp FROM picks WHERE length(c1) = {num_colors * 7 - 1}"
    ):
        count += 1
        orders = [[int(c) for c in o] for o in row[2].split(",")]
        # Convert to Jab [CAM02-UCS based]
        jab1 = to_jab(row[0].split(","))
        jab2 = to_jab(row[1].split(","))
        # Add cycle data
        for i in range(4):
            if i != row[3] - 1:
                cycle_data[num_colors].append(
                    np.array((jab1[orders[row[3] - 1]], jab2[orders[i]])).flatten()
                )
                cycle_targets[num_colors].append(0)
    cycle_data[num_colors] = np.array(cycle_data[num_colors])
    cycle_targets[num_colors] = np.array(cycle_targets[num_colors])
    min_count = min(min_count, count)
    print(num_colors, count)

conn.close()

6 10347
8 10371


## Construct, train, and evaluate ensemble instance

### Construct network

A fully-convolutional architecture is used, and weights are shared between different color cycle sizes. An homogenous ensemble of `ENSEMBLE_COUNT` models is constructed, to be trained in parallel using bootstrap aggregation (bagging) to divide the data into training and test sets.

### Split data into training and test sets

Bootstrap aggregation (bagging) is used to split the data into training and test sets, with a different data split used for each member of the ensemble. The `DATA_SPLIT_FRAC` sets the average fraction of data used in the training set (this is a statistical property, due to the stocastic nature of bagging). For each data split, all examples not in the training data set are placed in the test data set (which is not always the same size, again due to the stocastic nature of bagging).

In [9]:
def run_iteration():
    #
    # Construct network
    #
    input_a = {}
    input_b = {}
    model = {}
    predictions = {}
    scoring_model = {}

    reg = tf.keras.regularizers.l2(0.0001)

    conv_size = 5

    layer1 = Dense(units=5, activation="elu", activity_regularizer=reg, name="l1")
    layer2 = Dense(units=5, activation="elu", activity_regularizer=reg, name="l2")

    layer3 = SeparableConv1D(
        5,
        conv_size,
        padding="same",
        activation="elu",
        activity_regularizer=reg,
        name="l3",
    )
    layer4 = SeparableConv1D(
        3,
        conv_size,
        padding="same",
        activation="elu",
        activity_regularizer=reg,
        name="l4",
    )
    layer5 = SeparableConv1D(
        1,
        conv_size,
        padding="same",
        activation="elu",
        activity_regularizer=reg,
        name="l5",
    )

    for num_colors in ALL_NUM_COLORS:
        # Create network
        # One input per color set
        input_a[num_colors] = Input(shape=(3 * num_colors,))
        input_b[num_colors] = Input(shape=(3 * num_colors,))
        inputs_a = [
            input_a[num_colors][:, i * 3 : (i + 1) * 3] for i in range(num_colors)
        ]
        inputs_b = [
            input_b[num_colors][:, i * 3 : (i + 1) * 3] for i in range(num_colors)
        ]

        # Share layers between colors
        x_a = [layer1(i / 100) for i in inputs_a]
        x_b = [layer1(i / 100) for i in inputs_b]

        x_a = [layer2(i) for i in x_a]
        x_b = [layer2(i) for i in x_b]

        # Combine colors into sets
        x_a = tf.keras.layers.concatenate(
            [tf.keras.backend.expand_dims(i, 1) for i in x_a], axis=1
        )
        x_b = tf.keras.layers.concatenate(
            [tf.keras.backend.expand_dims(i, 1) for i in x_b], axis=1
        )

        # Share layers between color sets
        x_a = layer3(x_a)
        x_b = layer3(x_b)

        x_a = layer4(x_a)
        x_b = layer4(x_b)

        x_a = layer5(x_a)
        x_b = layer5(x_b)

        # Average outputs
        x_a = tf.math.reduce_mean(x_a, axis=1)
        x_b = tf.math.reduce_mean(x_b, axis=1)

        # Make conjoined network with output score
        layer_nm1 = Activation("sigmoid", name=f"score{num_colors}")
        x_a = layer_nm1(x_a)
        x_b = layer_nm1(x_b)
        predictions[num_colors] = tf.keras.layers.subtract([x_b, x_a])
        predictions[num_colors] = Activation("sigmoid", name=f"a{num_colors}")(
            predictions[num_colors]
        )

        # Compile model
        model[num_colors] = Model(
            inputs=[input_a[num_colors], input_b[num_colors]],
            outputs=predictions[num_colors],
        )
        model[num_colors].compile(loss="binary_crossentropy", metrics=["acc"])

        # For evaluation
        scoring_model[num_colors] = Model(inputs=input_a[num_colors], outputs=x_a)

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01, centered=True)
    modelc = Model(
        inputs=[i[nc] for nc in ALL_NUM_COLORS for i in (input_a, input_b)],
        outputs=[predictions[nc] for nc in ALL_NUM_COLORS],
    )
    modelc.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["acc"])

    #
    # Create training / test data splits
    #
    x_trains = {}
    y_trains = {}
    x_tests = {}
    y_tests = {}

    for num_colors in ALL_NUM_COLORS:
        # Split dataset (bagging)
        # The three pairs from each response are kept together
        idx = np.arange(min_count)
        train_idx = sklearn.utils.resample(
            idx,
            replace=True,
            n_samples=int(min_count * DATA_SPLIT_FRAC / (1 - 1 / np.e)),
        )
        test_idx = np.array([i for i in idx if i not in train_idx])
        train_idx = np.concatenate([train_idx * 3, train_idx * 3 + 1, train_idx * 3 + 2])
        test_idx = np.concatenate([test_idx * 3, test_idx * 3 + 1, test_idx * 3 + 2])
        x_trains[num_colors] = cycle_data[num_colors][train_idx]
        y_trains[num_colors] = cycle_targets[num_colors][train_idx]
        x_tests[num_colors] = cycle_data[num_colors][test_idx]
        y_tests[num_colors] = cycle_targets[num_colors][test_idx]

    #
    # Fit network
    #
    h = modelc.fit(
        list(
            itertools.chain(
                *[
                    [
                        x_trains[nc][:, : nc * 3],
                        x_trains[nc][:, nc * 3 :],
                    ]
                    for nc in ALL_NUM_COLORS
                ]
            )
        ),
        [y_trains[nc] for nc in ALL_NUM_COLORS],
        epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=0,
    )

    #
    # Evaluate network
    #
    eval_train = {}
    eval_test = {}
    for nc in ALL_NUM_COLORS:
        eval_train[nc] = model[nc].evaluate(
            [x_trains[nc][:, : nc * 3], x_trains[nc][:, nc * 3 :]],
            y_trains[nc],
            batch_size=8192,
            verbose=0,
        )[1]
        eval_test[nc] = model[nc].evaluate(
            [x_tests[nc][:, : nc * 3], x_tests[nc][:, nc * 3 :]],
            y_tests[nc],
            batch_size=8192,
            verbose=0,
        )[1]

    return eval_train, eval_test, scoring_model[ALL_NUM_COLORS[0]]

## Train ensemble

Iteratively train each instance of model ensemble. The train and test set accuracy are recorded, and the trained model weights are saved.

### Save model weights

Model weights are saved in the HDF5 format, to allow for loading by name. This is important, since it allows for loading the weights into a model with a different topology, e.g., a model that only works for a different color set size. While the model weights are independent of color set size, the actual model needs to be constructed with a fixed number of colors (at least using TensorFlow). Gzip compression is used on the HDF5 files, since this reduces their sizes by more than an order of magnitude.

In [10]:
t0 = time.time()

os.makedirs("weights", exist_ok=True)

train_acc = []
test_acc = []
weight_sum = 0

# Without this, TensorFlow will start complaining about retracing after the first couple iterations
tf.get_logger().setLevel(logging.ERROR)

for i in range(ENSEMBLE_COUNT):
    eval_train, eval_test, scoring_model = run_iteration()
    train_acc.append(eval_train)
    test_acc.append(eval_test)

    # Sum weights to check for reproducibility
    weight_sum += sum([np.sum(np.abs(w)) for w in scoring_model.get_weights()])

    # Save weights
    weight_file_name = f"weights/cycle_model_weights_{i:03d}.h5"
    scoring_model.save_weights(weight_file_name)
    with open(weight_file_name, "rb") as infile:
        with gzip.open(weight_file_name + ".gz", "wb") as outfile:
            shutil.copyfileobj(infile, outfile)
    os.remove(weight_file_name)

    if i == 0:
        param_count = scoring_model.count_params()
        print("Number of parameters per ensemble instance:", param_count)

print(f"Weight sum: {weight_sum:.5f}")
print(f"Training time: {time.time() - t0:.1f}s")

Number of parameters per ensemble instance: 167
Weight sum: 9349.75452
Training time: 22182.6s


In [11]:
for nc in ALL_NUM_COLORS:
    acc = [i[nc] for i in train_acc]
    print(f"train accuracy {nc}: {np.mean(acc):.5f} +/- {np.std(acc):.5f}")
    acc = [i[nc] for i in test_acc]
    print(f" test accuracy {nc}: {np.mean(acc):.5f} +/- {np.std(acc):.5f}")

train accuracy 6: 0.54394 +/- 0.00418
 test accuracy 6: 0.52585 +/- 0.00649
train accuracy 8: 0.54638 +/- 0.00483
 test accuracy 8: 0.52977 +/- 0.00582
